In [ ]:
# 导入必要的模块
import sys
import os
import time

src_dir = os.path.dirname(os.path.dirname(os.path.abspath(''))) + '/src'
sys.path.append(src_dir)

print(src_dir)

from datetime import date
from hashindex import HashIndex
from views import Views
from database import Database

In [ ]:
# 初始化视图和数据库连接
views = Views()
db = Database()

In [ ]:
# # 插入数据
# %run insert.py

In [ ]:
# 无索引查询

def query_without_index():
    db = Database()
    start_time = time.time()
    result = db.execute("""
        SELECT SUM(od.unit_price * od.quantity) 
        FROM OrderDetail od
        JOIN Orders o ON od.order_id = o.order_id
        WHERE o.customer_id = 'customer0'
    """)
    end_time = time.time()
    total_amount = result[0][0] if result else 0
    print(f"Query without index took {end_time - start_time:.4f} seconds")
    print(f"Total amount for customer0: {total_amount}")

print("\nQuery without any index:")
query_without_index()

In [ ]:
# 创建B-Tree索引
start_time = time.time()
%run btreeindex.py
end_time = time.time()
print(f"B-Tree Index creation took {end_time - start_time:.4f} seconds")

# 创建散列索引
start_time = time.time()
hash_index = HashIndex()
hash_index.build_index()
end_time = time.time()
print(f"Hash Index creation took {end_time - start_time:.4f} seconds")

In [ ]:
# 散列索引查询

def query_with_hash_index():
    start_time = time.time()
    hash_index = HashIndex()
    hash_index.build_index()
    order_ids = hash_index.search_orders('customer0')
    total_amount = hash_index.search_order_details(order_ids)
    end_time = time.time()
    print(f"Query with Hash Index took {end_time - start_time:.4f} seconds")
    print(f"Total amount for customer0: {total_amount}")

print("\nQuery with Hash Index:")
query_with_hash_index()

In [ ]:
def query_with_btree_index():
    db = Database()
    start_time = time.time()
    result = db.execute("""
        SELECT SUM(od.unit_price * od.quantity) 
        FROM OrderDetail od
        JOIN Orders o ON od.order_id = o.order_id
        WHERE o.customer_id = 'customer0'
    """)
    end_time = time.time()
    total_amount = result[0][0] if result else 0
    print(f"Query with B-Tree Index took {end_time - start_time:.4f} seconds")
    print(f"Total amount for customer0: {total_amount}")

print("\nQuery with B-Tree Index:")
query_with_btree_index()

In [ ]:
# 内连接查询

def query_with_inner_join():
    db = Database()
    start_time = time.time()
    result = db.execute("""
        SELECT SUM(od.unit_price * od.quantity) 
        FROM Orders o
        JOIN OrderDetail od ON o.order_id = od.order_id
        WHERE o.customer_id = 'customer0'
    """)
    end_time = time.time()
    total_amount = result[0][0] if result else 0
    print(f"Query with INNER JOIN took {end_time - start_time:.4f} seconds")
    print(f"Total amount for customer0: {total_amount}")

print("\nQuery with INNER JOIN:")
query_with_inner_join()

In [ ]:
def query_with_subquery():
    db = Database()
    start_time = time.time()
    result = db.execute("""
        SELECT SUM(od.unit_price * od.quantity) 
        FROM OrderDetail od
        WHERE od.order_id IN (
            SELECT o.order_id 
            FROM Orders o 
            WHERE o.customer_id = 'customer0'
        )
    """)
    end_time = time.time()
    total_amount = result[0][0] if result else 0
    print(f"Query with subquery took {end_time - start_time:.4f} seconds")
    print(f"Total amount for customer0: {total_amount}")

print("\nQuery with subquery:")
query_with_subquery()